# 도구를 통한 Strands Agents와 AgentCore Memory (장기 메모리)

## 소개

이 노트북은 AgentCore Memory와 Strands 프레임워크를 사용하여 **공유 장기 메모리를 가진 멀티 에이전트 시스템**을 구현하는 방법을 보여줍니다. 각 에이전트에 대한 전용 네임스페이스를 가진 공통 장기 메모리 저장소에 액세스하면서 여러 전문 에이전트가 함께 작업하는 방법을 탐구합니다.

### 튜토리얼 세부사항

| 정보                | 세부사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형        | 장기 대화형                                                                         |
| 에이전트 유형        | 여행 예약 비서                                                                 |
| 에이전트 프레임워크   | Strands Agents                                                                   |
| LLM 모델            | Anthropic Claude Sonnet 3.7                                                      |
| 튜토리얼 구성요소     | AgentCore 사용자 선호도 메모리 추출, 메모리 저장 및 검색을 위한 도구              |
| 예제 복잡도          | 중급                                                                             |

배울 내용:

- 장기 메모리 전략으로 공유 메모리 리소스 설정 방법
- 자체 메모리 네임스페이스에 액세스할 수 있는 전문 에이전트 생성
- 전문 에이전트에게 위임하는 코디네이터 에이전트 구현
- 에이전트 전문화를 위한 구조화된 메모리 네임스페이스 활용

## 시나리오 컨텍스트

이 예제에서는 다음과 같은 **여행 계획 시스템**을 만들어보겠습니다:
1. 여행 선호도와 기록의 장기 메모리를 가진 항공편 예약 비서
2. 숙박 선호도의 장기 메모리를 가진 호텔 예약 비서
3. 이러한 전문 에이전트들을 조율하는 여행 코디네이터

각 전문 에이전트는 공통 메모리 저장소 내에서 자체 네임스페이스에 액세스하여 시간이 지나면서 사용자 선호도에 대한 지속적인 이해를 구축할 수 있습니다. 이 접근 방식은 복잡한 도메인을 메모리 인프라를 공유하지만 자체 전문 영역을 유지하는 전문 에이전트로 나눠 구성할 수 있는 방법을 보여줍니다.

## 아키텍처
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 전제조건

이 튜토리얼을 실행하려면 다음이 필요합니다:
- Python 3.10+
- Amazon Bedrock AgentCore Memory 권한이 있는 AWS 자격 증명
- Amazon Bedrock AgentCore SDK

환경 설정과 공유 장기 메모리 리소스 생성을 시작해보겠습니다!

## 1단계: 환경 설정
이 노트북이 작동하도록 필요한 모든 라이브러리를 가져오고 클라이언트를 정의하는 것부터 시작해보겠습니다.

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
import time
from datetime import datetime
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("travel-assistant")

Amazon Bedrock 모델과 AgentCore에 대한 적절한 권한을 가진 지역을 정의합니다

In [ ]:
region = "us-west-2" # Replace with your AWS region
MODEL_ID= "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

## 2단계: 공유 메모리 리소스 생성
이 섹션에서는 각 에이전트에 대한 전용 네임스페이스를 가진 공통 장기 메모리 저장소를 생성합니다.

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

client = MemoryClient(region_name=region)
memory_name = "TravelAgent_LTM"
memory_id = None

In [ ]:
from botocore.exceptions import ClientError

try:
    print("Creating Memory with Long-Term Strategy...")
    # Create the memory resource with a single long-term memory strategy
    # The {actorId} placeholder will be dynamically replaced with the actual actor ID
    memory = client.create_memory_and_wait(
        name=memory_name,
        description="Travel Agent with Long-Term Memory",
        strategies=[{
            StrategyType.USER_PREFERENCE.value: {
                "name": "UserPreferences",
                "description": "Captures user preferences",
                "namespaces": ["travel/{actorId}/preferences"]
            }
        }],
        event_expiry_days=7,  # Short-term conversation expires after 7 days
        max_wait=300,
        poll_interval=10
    )

    # Extract and print the memory ID
    memory_id = memory['id']
    print(f"Memory created successfully with ID: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

### 장기 메모리 전략 이해

우리가 생성하는 메모리 리소스는 단일 사용자 선호도 전략으로 AgentCore Memory의 장기 메모리 기능을 사용합니다:

1. **사용자 선호도 메모리 전략**: 대화에서 언급된 사용자 선호도를 자동으로 추출하고 통합
2. **액터 기반 네임스페이스**: 네임스페이스 경로에 액터 ID를 사용하여 각 에이전트에 대한 별도 공간 생성
3. **메모리 지속성**: 만료되는 단기 메모리와 달리 추출된 선호도는 대화 만료 후에도 지속

`travel/{actorId}/preferences` 네임스페이스 패턴으로 각 전문 에이전트는 액터 ID에 기반한 고유한 네임스페이스를 가집니다:
- 항공편 에이전트 액세스: `travel/flight-user-TIMESTAMP/preferences`
- 호텔 에이전트 액세스: `travel/hotel-user-TIMESTAMP/preferences`

이를 통해 각 에이전트는 공통 메모리 인프라를 사용하면서도 자체 전문 지식을 유지할 수 있습니다.

### 에이전트 신원 설정

In [ ]:
# Create unique actor IDs for each specialized agent but share the session ID
flight_actor_id = f"flight-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
hotel_actor_id = f"hotel-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"travel-session-{datetime.now().strftime('%Y%m%d%H%M%S')}"
flight_namespace = f"travel/{flight_actor_id}/preferences"
hotel_namespace = f"travel/{hotel_actor_id}/preferences"

In [ ]:
# Import the necessary components
from strands import Agent, tool
from strands_tools.agent_core_memory import AgentCoreMemoryToolProvider

### 3단계: 메모리 훅 프로바이더 생성

이 단계에서는 메모리 작업을 자동화하는 사용자 정의 `MemoryHookProvider` 클래스를 정의합니다. 훅은 에이전트의 실행 생명주기의 특정 지점에서 실행되는 특수 함수입니다. 우리가 만드는 메모리 훅은 하나의 주요 기능을 제공합니다:

1. **메모리 저장**: 에이전트가 응답한 후 새로운 대화를 저장

이는 수동 관리 없이 원활한 메모리 경험을 만들어줍니다.

In [ ]:
class MemoryHookProvider(HookProvider):
    """Hook provider for automatic memory management"""
    
    def __init__(self, memory_id: str, client: MemoryClient, actor_id: str, session_id: str):
        self.memory_id = memory_id
        self.client = client
        self.actor_id = actor_id
        self.session_id = session_id
    
    def save_memories(self, event: AfterInvocationEvent):
        """Save conversation after agent response"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2:
                # Get last user and assistant messages
                user_msg = None
                assistant_msg = None
                
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not assistant_msg:
                        assistant_msg = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not user_msg and "toolResult" not in msg["content"][0]:
                        user_msg = msg["content"][0]["text"]
                        break
                
                if user_msg and assistant_msg:
                    # Save conversation
                    self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=self.actor_id,
                        session_id=self.session_id,
                        messages=[(user_msg, "USER"), (assistant_msg, "ASSISTANT")]
                    )
                    logger.info("Saved conversation to memory")
                    
        except Exception as e:
            logger.error(f"Failed to save memories: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        """Register memory hooks"""
        registry.add_callback(AfterInvocationEvent, self.save_memories)
        logger.info("Memory hooks registered")

### 메모리 액세스를 가진 전문 에이전트 생성

다음으로, 전문 에이전트를 위한 시스템 프롬프트를 정의하겠습니다.

In [ ]:
# System prompt for the hotel booking specialist
HOTEL_BOOKING_PROMPT = f"""You are a hotel booking assistant. Help customers find hotels, make reservations, and answer questions about accommodations and amenities. 
Provide clear information about availability, pricing, and booking procedures in a friendly, helpful manner.Ask max two questions per turn. Keep the messages short, don't overwhelm the customer."""

# System prompt for the flight booking specialist
FLIGHT_BOOKING_PROMPT = f"""You are a flight booking assistant. Help customers find flights, make reservations, and answer questions about airlines, routes, and travel policies. 
Provide clear information about flight availability, pricing, schedules, and booking procedures in a friendly, helpful manner.Ask max two questions per turn. Keep the messages short, don't overwhelm the customer."""

### 에이전트 도구 구현
이제 코디네이터 에이전트가 사용할 수 있는 도구로서 전문 에이전트를 구현해보겠습니다:

In [ ]:
@tool
def flight_booking_assistant(query: str) -> str:
    """
    Process and respond to flight booking queries.

    Args:
        query: A flight-related question about bookings, schedules, airlines, or travel policies

    Returns:
        Detailed flight information, booking options, or travel advice
    """
    try:
        provider_flight = AgentCoreMemoryToolProvider(
            memory_id=memory_id,      # Required
            actor_id=flight_actor_id, # Required
            session_id=session_id,    # Required
            max_results=10,           # Optional
            namespace=flight_namespace
        )
        
        flight_memory_hooks = MemoryHookProvider(
            memory_id=memory_id,
            client=client,
            actor_id=flight_actor_id,
            session_id=session_id
        )

        flight_agent = Agent(tools=provider_flight.tools,
                             hooks=[flight_memory_hooks],
                             model=MODEL_ID,
                             system_prompt=FLIGHT_BOOKING_PROMPT
                             )

        # Call the agent and return its response
        response = flight_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in flight booking assistant: {str(e)}"

@tool
def hotel_booking_assistant(query: str) -> str:
    """
    Process and respond to hotel booking queries.

    Args:
        query: A hotel-related question about accommodations, amenities, or reservations

    Returns:
        Detailed hotel information, booking options, or accommodation advice
    """
    try:
        provider_hotel = AgentCoreMemoryToolProvider(
            memory_id=memory_id,      
            actor_id=hotel_actor_id, 
            session_id=session_id,    
            max_results=10,           
            namespace=hotel_namespace
        )

        hotel_memory_hooks = MemoryHookProvider(
            memory_id=memory_id,
            client=client,
            actor_id=hotel_actor_id,
            session_id=session_id
        )

        hotel_booking_agent = Agent(tools=provider_hotel.tools, hooks=[hotel_memory_hooks],model=MODEL_ID, system_prompt=HOTEL_BOOKING_PROMPT)
        
        # Call the agent and return its response
        response = hotel_booking_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in hotel booking assistant: {str(e)}"

### 코디네이터 에이전트 생성

마지막으로, 이러한 전문 도구들 간에 조정을 담당하는 메인 여행 계획 에이전트를 만들어보겠습니다:

In [ ]:
# System prompt for the coordinator agent
TRAVEL_AGENT_SYSTEM_PROMPT = """
You are a comprehensive travel planning assistant that coordinates between specialized tools:
- For flight-related queries (bookings, schedules, airlines, routes) → Use the flight_booking_assistant tool
- For hotel-related queries (accommodations, amenities, reservations) → Use the hotel_booking_assistant tool
- For complete travel packages → Use both tools as needed to provide comprehensive information
- For general travel advice or simple travel questions → Answer directly

Each agent will have its own memory in case the user asks about historic data.
When handling complex travel requests, coordinate information from both tools to create a cohesive travel plan.
Provide clear organization when presenting information from multiple sources. \
Ask max two questions per turn. Keep the messages short, don't overwhelm the customer.
"""

In [ ]:
travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    model=MODEL_ID,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

## 멀티 에이전트 메모리 시스템 테스트

여행 계획 시나리오로 멀티 에이전트 시스템을 테스트해보겠습니다:

### 선택사항: 항공편 예약 비서의 장기 메모리 하이드레이트
항공편 예약 에이전트를 하이드레이트하려면 다음 셀의 주석을 해제하세요.

In [ ]:
"""flight_previous_messages = [
    ("Hi, I'm Sarah", "USER"),
    ("Hello Sarah! Welcome to FlightAssist. How can I help you with your travel plans today?", "ASSISTANT"),
    ("I'm looking to book a flight from New York to London sometime next month.", "USER"),
    ("I'd be happy to help you find flights from New York to London for next month. Do you have specific dates in mind, or are you flexible?", "ASSISTANT"),
    ("I'm thinking around the 15th to the 25th, but I can be a bit flexible.", "USER"),
    ("Great! That gives us some room to find the best options. Do you have any preferences regarding airlines or flight times?", "ASSISTANT"),
    ("I definitely prefer direct flights if possible. I really don't like layovers.", "USER"),
    ("I completely understand your preference for direct flights. Layovers can be inconvenient. Fortunately, there are several airlines offering direct flights between New York and London, including British Airways, American Airlines, Delta, and Virgin Atlantic.", "ASSISTANT"),
    ("That's good to hear. I've had good experiences with British Airways in the past.", "USER"),
    ("British Airways does offer excellent service on transatlantic routes. I'll keep that in mind when searching for options. Do you have any seating preferences or other requirements for your flight?", "ASSISTANT"),
    ("I always try to get an aisle seat. I like being able to get up without disturbing others, especially on long flights.", "USER"),
    ("An aisle seat is a great choice for long-haul flights like New York to London. I'll note your preference for aisle seating. Would you prefer to fly in the morning, afternoon, or evening?", "ASSISTANT"),
    ("I prefer overnight flights for long journeys. It helps me adjust to the time difference better.", "USER"),
    ("Overnight flights are indeed a smart choice for eastbound transatlantic travel. They allow you to arrive in London in the morning and help minimize jet lag. British Airways, Delta, and American all offer evening departures from New York that arrive in London the next morning.", "ASSISTANT"),
    ("Perfect! And I'm also wondering about baggage allowances since I'll be staying for about a week.", "USER"),
    ("For a week-long trip, most travelers find that a standard checked bag plus a carry-on is sufficient. British Airways typically allows one free checked bag on transatlantic flights in economy class, plus a carry-on and personal item. Would you like me to check the specific allowances for your preferred dates?", "ASSISTANT")
]

print("\nHydrating memories with previous conversations...")

# Save the conversation history to short-term memory
initial = client.create_event(
    memory_id=memory_id,
    actor_id=flight_actor_id,
    session_id=session_id,
    messages=flight_previous_messages,
)
print("✓ Conversation saved in short term memory")"""

In [ ]:
travel_agent("Hello, I would like to book a trip from LA to Madrid. From July 1 to August 2.")

In [ ]:
travel_agent("I prefer direct flights with Iberia")

In [ ]:
travel_agent("I would like a flight in the morning, in economy")

In [ ]:
travel_agent("I would like to fly from SNA, and return 15 days later")

## 메모리 지속성 테스트

메모리 시스템이 올바르게 작동하는지 테스트하기 위해 여행 에이전트의 새 인스턴스를 생성하고 이전에 저장된 정보에 액세스할 수 있는지 확인해보겠습니다:

In [ ]:
time.sleep(60) # Let's give the memory some time to process the events..
# Create a new instance of the travel agent
new_travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    model=MODEL_ID,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

# Ask about previous conversations
new_travel_agent("Can you remind me about flight preferences?")

## 요약

이 노트북에서 보여준 내용:

1. 여러 에이전트를 위한 공유 메모리 리소스 생성 방법
2. 메모리 액세스를 가진 도구로서 전문 에이전트 구현 방법
3. 대화 컨텍스트를 유지하면서 여러 에이전트 간 조정 방법
4. 다른 에이전트 인스턴스 간 메모리 지속 방법

공유 메모리를 가진 이 멀티 에이전트 아키텍처는 일관된 사용자 경험을 유지하면서 전문 도메인을 처리할 수 있는 복잡한 대화형 AI 시스템을 구축하는 강력한 접근 방식을 제공합니다.

## 정리
이 노트북에서 사용한 리소스를 정리하기 위해 메모리를 삭제해보겠습니다.

In [ ]:
# client.delete_memory_and_wait(
#        memory_id = memory_id,
#        max_wait = 300,
#        poll_interval =10
#)